In [51]:
import pandas as pd
import matplotlib as plt
import numpy as np

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
pre_model = pd.read_csv('/content/drive/MyDrive/CAPSTONE/ML/premodeldf.csv')

In [54]:
pre_model

,task,time_hour,tasksum
0,jemur,7,4
1,jogging,7,3
2,olahraga,7,2
3,senam taichi,7,2
4,bejemur,7,2
...,...,...,...
106,tidur,19,2
107,minum vitamin,19,1
108,makan malam,19,1
109,tidur malam,20,1


In [55]:
taskindex = pd.DataFrame(pre_model.task.unique(), columns=['task']).reset_index()
taskindex.rename({'index':'task_idx'}, axis=1, inplace=True)
taskindex

,task_idx,task
0,0,jemur
1,1,jogging
2,2,olahraga
3,3,senam taichi
4,4,bejemur
5,5,senam poco poco
6,6,gosok gigi
7,7,main teka teki silang
8,8,jalan pagi
9,9,siram tanam


In [56]:
pre_model_input = pre_model.merge(taskindex, on='task')

pre_model_input

,task,time_hour,tasksum,task_idx
0,jemur,7,4,0
1,jemur,8,4,0
2,jogging,7,3,1
3,jogging,8,1,1
4,jogging,16,1,1
...,...,...,...,...
106,makan,18,1,50
107,membersikan telinga,18,1,51
108,tidur,19,2,52
109,tidur malam,20,1,53


In [57]:
model_input = pre_model_input[['task_idx', 'time_hour', 'tasksum']]
model_input

,task_idx,time_hour,tasksum
0,0,7,4
1,0,8,4
2,1,7,3
3,1,8,1
4,1,16,1
...,...,...,...
106,50,18,1
107,51,18,1
108,52,19,2
109,53,20,1


In [58]:
#model_input.to_csv('/content/drive/MyDrive/CAPSTONE/ML/modelinput.csv', index=False)
#model_input.to_excel('/content/drive/MyDrive/CAPSTONE/ML/modelinput.xlsx', index=False)

In [59]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(model_input, test_size=0.2, random_state=42)

In [60]:
train.head()

,task_idx,time_hour,tasksum
26,12,9,1
22,9,16,3
31,15,8,1
47,22,18,1
76,32,11,1


In [61]:
model_input.shape

(111, 3)

# MODEL

In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
import tensorflow as tf
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [63]:
n_hour = len(model_input.time_hour.value_counts())
n_task = len(model_input.task_idx.value_counts())
print(n_hour, n_task)

15 54


In [64]:
hourinput = Input(shape=[1], name='hour-input')
hourvector = Flatten(name='hour-flatten')(hourinput)

taskinput = Input(shape=[1], name='task-input')
taskembedding = Embedding(n_task+1, 64, name='task-embedding')(taskinput)
taskvector = Flatten(name='task-flatten')(taskembedding)

conc = Concatenate()([hourvector, taskvector])

fc1 = Dense(64, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
fc3 = Dense(32, activation='relu')(fc2)
out = Dense(1)(fc3)

In [65]:
conc

<KerasTensor: shape=(None, 65) dtype=float32 (created by layer 'concatenate_1')>

In [66]:
model = Model([hourinput, taskinput], out)
model.compile('adam', 'mean_squared_error')

In [67]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 task-input (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 hour-input (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 task-embedding (Embedding)     (None, 1, 64)        3520        ['task-input[0][0]']             
                                                                                                  
 hour-flatten (Flatten)         (None, 1)            0           ['hour-input[0][0]']             
                                                                                            

In [68]:
time = np.array(train.time_hour)
tasks = np.array(train.task_idx)
sums = np.array(train['tasksum'].values)

In [69]:
history = model.fit(
    x = [time,tasks],
    y = sums,
    epochs=500,
    verbose=1, 
    )

Epoch 1/500
3/3 [==============================] - 1s 6ms/step - loss: 2.2817
Epoch 2/500
3/3 [==============================] - 0s 10ms/step - loss: 1.5034
Epoch 3/500
3/3 [==============================] - 0s 7ms/step - loss: 1.2045
Epoch 4/500
3/3 [==============================] - 0s 8ms/step - loss: 1.1108
Epoch 5/500
3/3 [==============================] - 0s 6ms/step - loss: 1.1833
Epoch 6/500
3/3 [==============================] - 0s 5ms/step - loss: 1.1966
Epoch 7/500
3/3 [==============================] - 0s 7ms/step - loss: 1.1488
Epoch 8/500
3/3 [==============================] - 0s 5ms/step - loss: 1.1071
Epoch 9/500
3/3 [==============================] - 0s 5ms/step - loss: 1.0857
Epoch 10/500
3/3 [==============================] - 0s 6ms/step - loss: 1.0754
Epoch 11/500
3/3 [==============================] - 0s 5ms/step - loss: 1.0711
Epoch 12/500
3/3 [==============================] - 0s 6ms/step - loss: 1.0568
Epoch 13/500
3/3 [==============================] - 0s 6ms/s

In [70]:
model.evaluate([test.time_hour,test.task_idx], np.array(test['tasksum']))


1/1 [==============================] - 0s 265ms/step - loss: 1.5647


1.5647168159484863

In [71]:
# Save Model
SAVED_PATH = '/content/drive/MyDrive/CAPSTONE/ML/modelterakhir.h5'


In [72]:
model.save(SAVED_PATH, save_format='h5')

# OTW TFLite

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [73]:
import pathlib
import tensorflow as tf

In [74]:
load_model = tf.keras.models.load_model('/content/drive/MyDrive/CAPSTONE/ML/modelterakhir.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(load_model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp2nrd6ksu/assets


INFO:tensorflow:Assets written to: /tmp/tmp2nrd6ksu/assets


In [75]:
tflite_model_files = pathlib.Path('/content/drive/MyDrive/CAPSTONE/ML/modelterakhir.tflite')
tflite_model_files.write_bytes(tflite_model)

46876

# Prediction Test

In [76]:
saved_model = tf.keras.models.load_model(SAVED_PATH)

In [77]:
task_data = np.array(list(set(model_input.task_idx)))
task_data[:3]

h = 12
hour = np.array([h for i in range(len(task_data))])
hour[:3]

predictions = model.predict([hour, task_data])

predictions = np.array([a[0] for a in predictions])

recommend = (-predictions).argsort()[:10]

recommend

array([40, 43, 41,  9,  0, 38, 45, 52, 10,  6])

In [78]:
predictions[recommend]

array([4.8454957, 4.626925 , 3.4047072, 3.1508732, 2.2427182, 2.2195444,
       1.8948705, 1.8353958, 1.6804247, 1.5787979], dtype=float32)

In [84]:
model.predict([np.array([12,12,12,12]), np.array([40, 43, 41,  0])])

array([[4.8454957],
       [4.626925 ],
       [3.4047072],
       [2.2427182]], dtype=float32)